In [ ]:
!pip install llama-index
!pip install openai
!pip install llama-index-retrievers-bm25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 상담 기록 기반 연속성 높은 대화 서비스 구축

##### LLM 기반 쿼리 수정

In [ ]:
from openai import OpenAI

OPENAI_API_KEY= # OpenAI API Key
model_name="gpt-3.5-turbo"

client = OpenAI(
    api_key=OPENAI_API_KEY,
)

In [ ]:
from string import Template

prompt = Template("""
사용자의 질문에서 모호한 부분을 수정하고 더 필요할 것 같은 정보를 추가하여서 검색 모델에게 입력할 질문을 다시 구성해줘.

질문: $query
""")

In [ ]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt.substitute(query="저번에 취소했던 항공편을 다시 예약할 수 있을까요?"),
        }
    ],
    model=model_name,
)

print(chat_completion.choices[0].message.content)

수정된 질문: 저번에 취소한 항공편은 어떻게 다시 예약할 수 있을까요? 추가로 필요한 정보: 항공편 번호나 출발지-도착지, 날짜 및 시간과 같은 예약에 필요한 세부 정보를 알려주세요.


In [ ]:
def query_rewrite(query):
  chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt.substitute(query=query),
        }
    ],
    model=model_name,
  )

  return chat_completion.choices[0].message.content

##### llama index를 사용하여 이전 대화 기록 가져오기

In [ ]:
import os
from llama_index.llms.openai import OpenAI

os.environ['OPENAI_API_KEY'] = # OpenAI API Key


In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

## settings 에 llm을 등록하여 llama index의 다른 기능들에서 llm을 사용할 수 있다
Settings.llm = OpenAI(temperature=0.3, model="gpt-3.5-turbo")

folder_path = '/content/drive/MyDrive/데이터/part3_ch01_02'
documents = SimpleDirectoryReader(folder_path).load_data()
index = VectorStoreIndex.from_documents(
    documents,
)

In [ ]:
from llama_index.retrievers.bm25 import BM25Retriever

vector_retriever = index.as_retriever(similarity_top_k=2)


In [ ]:
docs = await vector_retriever.aretrieve("저번에 취소했던 항공편을 다시 예약할 수 있을까요?")
for doc in docs:
  print(f"Score: {doc.get_score()}")
  print(doc.get_content())
  print("\n\n")

Score: 0.8241169395309783
그 정보를 바탕으로 취소 절차에 대해 안내드리겠습니다."
            },
            {
                "role": "user",
                "content": "대한항공 오후 1:30분 출발 비행기요"
            },
            {
                "role": "assistant",
                "content": "대한항공 인천국제공항에서 존 F. 케네디 국제공항으로 가는 오후 1:30분 출발 항공편을 취소하시길 원하신다면, 대한항공 고객 서비스 센터에 연락하셔서 예약 취소를 요청하시면 됩니다. 대한항공 웹사이트나 모바일 앱을 통해서도 취소가 가능합니다.\n\n취소 수수료 및 환불 규정은 예약 시 선택하신 티켓 유형에 따라 다를 수 있으니, 이 점도 확인하시면 좋을 것 같습니다.\n\n더 도움이 필요하시거나 추가 정보가 필요하시면 언제든 문의해주세요!"
            }
        ]
    },
    {
        "id": 3,
        "name": "Alice Lee",
        "content": [
            {
                "role": "User",
                "content": "내일 오전 1시에 라가디아 공항으로 출발하는 비행편을 취소하고 싶어요"
            },
            {
                "role": "assistant",
                "content": "알겠습니다. 내일 오전 1시에 라가디아 공항으로 출발하는 비행편을 취소하는 것이 맞으시겠군요. 사용자님의 요청을 처리하기 위해 취소 정책과 환불 정책을 확인해야 합니다. 귀하의 항공편이 어느 항공사에서 예약되었는지 알려주실 수 있나요? 그렇게 하면 정확한 취소 절차와 환불 여부를 안내해 드릴 수 있습니다."
 

In [ ]:
docs = await vector_retriever.aretrieve(query_rewrite("저번에 취소했던 항공편을 다시 예약할 수 있을까요?"))
for doc in docs:
  print(f"Score: {doc.get_score()}")
  print(doc.get_content())
  print("\n\n")


Score: 0.8368998952268338
그 정보를 바탕으로 취소 절차에 대해 안내드리겠습니다."
            },
            {
                "role": "user",
                "content": "대한항공 오후 1:30분 출발 비행기요"
            },
            {
                "role": "assistant",
                "content": "대한항공 인천국제공항에서 존 F. 케네디 국제공항으로 가는 오후 1:30분 출발 항공편을 취소하시길 원하신다면, 대한항공 고객 서비스 센터에 연락하셔서 예약 취소를 요청하시면 됩니다. 대한항공 웹사이트나 모바일 앱을 통해서도 취소가 가능합니다.\n\n취소 수수료 및 환불 규정은 예약 시 선택하신 티켓 유형에 따라 다를 수 있으니, 이 점도 확인하시면 좋을 것 같습니다.\n\n더 도움이 필요하시거나 추가 정보가 필요하시면 언제든 문의해주세요!"
            }
        ]
    },
    {
        "id": 3,
        "name": "Alice Lee",
        "content": [
            {
                "role": "User",
                "content": "내일 오전 1시에 라가디아 공항으로 출발하는 비행편을 취소하고 싶어요"
            },
            {
                "role": "assistant",
                "content": "알겠습니다. 내일 오전 1시에 라가디아 공항으로 출발하는 비행편을 취소하는 것이 맞으시겠군요. 사용자님의 요청을 처리하기 위해 취소 정책과 환불 정책을 확인해야 합니다. 귀하의 항공편이 어느 항공사에서 예약되었는지 알려주실 수 있나요? 그렇게 하면 정확한 취소 절차와 환불 여부를 안내해 드릴 수 있습니다."
 

##### 3. 이전 상담 기록을 바탕으로 상담 이어나가기

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

from string import Template

prompt_rewrite = Template("""
사용자의 질문에서 모호한 부분을 수정하고 더 필요할 것 같은 정보를 추가하여서 검색 모델에게 입력할 질문을 다시 구성해줘.

질문: $query
""")

prompt_answer = Template("""
주어진 대화 기록을 기반으로 사용자의 질문에 대해 친절하게 답변해줘.

대화 기록: $information

질문: $query

""")

class ChatBotWithHistory:
  def __init__(self, history_root):
    self.history_root = history_root

    documents = SimpleDirectoryReader(history_root).load_data()
    index = VectorStoreIndex.from_documents(
        documents,
    )
    self.vector_retriever = index.as_retriever(similarity_top_k=1)

    self.psgs = ""
    self.rewrite = ""
    self.answer = ""

  async def query(self, query):
    # await self.query_rewrite(query)

    await self.retrieve(query)
    print(prompt_answer.substitute(query=query, information=self.psgs))
    answer = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": prompt_answer.substitute(query=query, information=self.psgs),
          }
      ],
      model=model_name,
    )

    return answer.choices[0].message.content

  async def retrieve(self, query):
    docs = await self.vector_retriever.aretrieve(query)

    psgs = ""
    for doc in docs:
      psgs += doc.get_content() + "\n\n"
    self.psgs = psgs

  async def query_rewrite(self, query):
    chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": prompt_rewrite.substitute(query=query),
          }
      ],
      model=model_name,
    )
    self.rewrite = chat_completion.choices[0].message.content

In [ ]:
chatBotWithHistory = ChatBotWithHistory('/content/drive/MyDrive/데이터/part3_ch01_02')

In [ ]:
await chatBotWithHistory.query("저번에 취소했던 항공편에 대한 상세 내용 알려주세요. 몇시 출발이었죠?")


주어진 대화 기록을 기반으로 사용자의 질문에 대해 친절하게 답변해줘.

대화 기록: 예를 들어, 오후 1:30 혹은 오후 5:00 등 원하시는 시간을 알려주시면 상세 예약 절차를 안내해 드리겠습니다."},
        {"role": "user", "content": "가장 빠른 시간대로 예약해 주세요"},
        {"role": "assistant",
         "content": "가장 빠른 시간대인 대한항공의 오후 1:30 비행기(INC → JFK)로 예약을 진행하겠습니다. 이 비행은 오늘 인천국제공항에서 출발해서 존 F. 케네디 국제공항에 오후 3:30에 도착합니다.\n\n저는 실제로 예약을 처리할 수 없으므로, 예약을 완료하시려면 대한항공의 공식 웹사이트나 가까운 여행 대리점을 통해 직접 예약하시거나 전화로 문의해 주시기 바랍니다. 필요한 개인 정보 및 결제 정보를 준비하시는 것이 좋습니다.\n\n더 도움이 필요하신 점 있으시면 알려주세요!"},
        {"role": "user", "content": "전화는 어느 번호로 해야 하나요?"},
        {"role": "assistant",
         "content": "제가 실제 전화번호 정보를 제공할 수 없습니다만, 대한항공의 예약이나 문의를 위해서는 대한항공 고객 서비스 센터에 연락하시는 것이 좋습니다. 일반적으로 대한항공의 고객 서비스 센터 번호는 공식 웹사이트에서 확인하실 수 있습니다. 웹사이트에 접속하시거나 인터넷 검색을 통해 '대한항공 고객 서비스 센터'로 검색하면 연락처를 쉽게 찾을 수 있습니다.\n\n필요한 도움을 받으시길 바랍니다! 추가로 도움이 필요하시면 언제든지 말씀해주세요."}]
    },
    {
        "id": 2,
        "name": "Jane Smith",
        "content": [
            {
                "role": "user",
                "conten

'안녕하세요! 이전에 취소하셨던 항공편은 대한항공의 오후 1:30분 출발 항공편이었습니다. 이 비행은 인천국제공항에서 존 F. 케네디 국제공항으로 운항되는 것으로 기억합니다. 해당 항공편에 대한 취소 절차나 환불에 대해 도움이 필요하시다면 대한항공 고객 서비스 센터에 문의하셔서 상세 내용을 확인하시길 권장드립니다. 도움이 필요하시면 언제든지 물어보세요!'

In [ ]:
await chatBotWithHistory.query("뉴욕으로 가는 항공편 예약했는데요. 몇시까지 공항에 도착해야 할까요?")


주어진 대화 기록을 기반으로 사용자의 질문에 대해 친절하게 답변해줘.

대화 기록: 예를 들어, 오후 1:30 혹은 오후 5:00 등 원하시는 시간을 알려주시면 상세 예약 절차를 안내해 드리겠습니다."},
        {"role": "user", "content": "가장 빠른 시간대로 예약해 주세요"},
        {"role": "assistant",
         "content": "가장 빠른 시간대인 대한항공의 오후 1:30 비행기(INC → JFK)로 예약을 진행하겠습니다. 이 비행은 오늘 인천국제공항에서 출발해서 존 F. 케네디 국제공항에 오후 3:30에 도착합니다.\n\n저는 실제로 예약을 처리할 수 없으므로, 예약을 완료하시려면 대한항공의 공식 웹사이트나 가까운 여행 대리점을 통해 직접 예약하시거나 전화로 문의해 주시기 바랍니다. 필요한 개인 정보 및 결제 정보를 준비하시는 것이 좋습니다.\n\n더 도움이 필요하신 점 있으시면 알려주세요!"},
        {"role": "user", "content": "전화는 어느 번호로 해야 하나요?"},
        {"role": "assistant",
         "content": "제가 실제 전화번호 정보를 제공할 수 없습니다만, 대한항공의 예약이나 문의를 위해서는 대한항공 고객 서비스 센터에 연락하시는 것이 좋습니다. 일반적으로 대한항공의 고객 서비스 센터 번호는 공식 웹사이트에서 확인하실 수 있습니다. 웹사이트에 접속하시거나 인터넷 검색을 통해 '대한항공 고객 서비스 센터'로 검색하면 연락처를 쉽게 찾을 수 있습니다.\n\n필요한 도움을 받으시길 바랍니다! 추가로 도움이 필요하시면 언제든지 말씀해주세요."}]
    },
    {
        "id": 2,
        "name": "Jane Smith",
        "content": [
            {
                "role": "user",
                "conten

'항공사나 공항에 따라 다르지만, 일반적으로 국제선 비행기를 탑승할 경우 최소 2시간 전에 공항에 도착하시는 것이 권장됩니다. 따라서 대한항공 오후 1:30분 출발 항공편의 경우, 오후 11:30분까지는 공항에 도착하는 것이 좋습니다. 하지만 정확한 시간은 대한항공이나 해당 공항의 운영 규정을 확인하는 것이 가장 확실할 것입니다. 예정 시간보다 조금 빨리 도착하여 여유롭게 체크인을 진행하시길 추천드립니다. 도움이 더 필요하시면 언제든지 물어보세요. 감사합니다.'